In [1]:
push!(LOAD_PATH,".")
push!(LOAD_PATH,"../..")

8-element Vector{String}:
 "/airlab"
 "/airlab/src/mdma_greedy"
 "/env"
 "@"
 "@v#.#"
 "@stdlib"
 "."
 "../.."

In [2]:
using Revise
using POMDPs
using DiscreteValueIteration
using POMDPTools
# using Dates
# using Profile

In [3]:
using MDMA

┌ Info: Precompiling MDMA [top-level]
└ @ Base loading.jl:1664


Test Summary: | Pass  Total  Time
detectTarget  |   30     30  0.4s
Test Summary:        |Time
single_robot_planner | None  0.1s
Test Summary:   |Time
test_file_parse | None  0.0s


In [4]:
# Set model parameters
focal_length = [19.99, 19.99]#mm
resolution = [3840.0,2160.0]
lens_dim = [35.00,24.00] #mm
pitch = -pi/4
sensor = PinholeCameraModel(focal_length, resolution, lens_dim, 0.0, pitch) 
#sensor = ViewConeSensor(pi/2, 3);
horizon = 20
grid = Grid(10,10, horizon);

# Setup initial distribution of targets
targets = Vector{Target}(undef, 0);
push!(targets, Target(grid.width-3,grid.height-3,0));
push!(targets, Target(grid.width-4,grid.height-2,0));
push!(targets, Target(grid.width-5,grid.height-3,0));

target_trajectories = MDMA.generate_target_trajectories(grid, horizon, targets);

move_dist = 3;
model = MDMA.SingleRobotMultiTargetViewCoverageProblem(grid, sensor, horizon, target_trajectories, move_dist);

In [5]:
policy = MDMA.solve_single_robot(model)


[Iteration 1   ] residual:       1.81 | iteration runtime:   4064.881 ms, (      4.06 s total)
[Iteration 2   ] residual:          0 | iteration runtime:   4026.652 ms, (      8.09 s total)


ValueIterationPolicy:
 MDPState(UAVState(1, 1, :E), 1, 20, nothing) -> MDPState(UAVState(2, 2, :SE), 2, 20, nothing)
 MDPState(UAVState(1, 2, :E), 1, 20, nothing) -> MDPState(UAVState(2, 2, :SE), 2, 20, nothing)
 MDPState(UAVState(1, 3, :E), 1, 20, nothing) -> MDPState(UAVState(2, 2, :SE), 2, 20, nothing)
 MDPState(UAVState(1, 4, :E), 1, 20, nothing) -> MDPState(UAVState(2, 2, :SE), 2, 20, nothing)
 MDPState(UAVState(1, 5, :E), 1, 20, nothing) -> MDPState(UAVState(3, 3, :SE), 2, 20, nothing)
 MDPState(UAVState(1, 6, :E), 1, 20, nothing) -> MDPState(UAVState(3, 4, :NE), 2, 20, nothing)
 MDPState(UAVState(1, 7, :E), 1, 20, nothing) -> MDPState(UAVState(3, 5, :NE), 2, 20, nothing)
 MDPState(UAVState(1, 8, :E), 1, 20, nothing) -> MDPState(UAVState(1, 6, :NE), 2, 20, nothing)
 MDPState(UAVState(1, 9, :E), 1, 20, nothing) -> MDPState(UAVState(1, 7, :NE), 2, 20, nothing)
 MDPState(UAVState(1, 10, :E), 1, 20, nothing) -> MDPState(UAVState(1, 8, :NE), 2, 20, nothing)
 MDPState(UAVState(2, 1, :E

In [6]:
path = compute_path(model, policy, MDPState(UAVState(1,1,:N), model.horizon))

20-element Vector{MDPState}:
 MDPState(UAVState(1, 1, :N), 1, 20, nothing)
 MDPState(UAVState(2, 3, :NE), 2, 20, nothing)
 MDPState(UAVState(2, 2, :E), 3, 20, nothing)
 MDPState(UAVState(2, 2, :NE), 4, 20, nothing)
 MDPState(UAVState(2, 2, :E), 5, 20, nothing)
 MDPState(UAVState(2, 2, :NE), 6, 20, nothing)
 MDPState(UAVState(2, 2, :E), 7, 20, nothing)
 MDPState(UAVState(2, 2, :NE), 8, 20, nothing)
 MDPState(UAVState(2, 2, :E), 9, 20, nothing)
 MDPState(UAVState(2, 2, :NE), 10, 20, nothing)
 MDPState(UAVState(2, 1, :E), 11, 20, nothing)
 MDPState(UAVState(1, 1, :NE), 12, 20, nothing)
 MDPState(UAVState(2, 1, :E), 13, 20, nothing)
 MDPState(UAVState(1, 1, :NE), 14, 20, nothing)
 MDPState(UAVState(2, 2, :E), 15, 20, nothing)
 MDPState(UAVState(2, 2, :NE), 16, 20, nothing)
 MDPState(UAVState(2, 2, :E), 17, 20, nothing)
 MDPState(UAVState(2, 2, :NE), 18, 20, nothing)
 MDPState(UAVState(2, 2, :E), 19, 20, nothing)
 MDPState(UAVState(2, 2, :NE), 20, 20, nothing)

In [7]:
draw_frames(RenderConf(50, 4, true, true), model, path)

Writing to output/01.png
Writing to output/02.png
Writing to output/03.png
Writing to output/04.png
Writing to output/05.png
Writing to output/06.png
Writing to output/07.png
Writing to output/08.png
Writing to output/09.png
Writing to output/10.png
Writing to output/11.png
Writing to output/12.png
Writing to output/13.png
Writing to output/14.png
Writing to output/15.png
Writing to output/16.png
Writing to output/17.png
Writing to output/18.png
Writing to output/19.png
Writing to output/20.png


In [10]:
using PyPlot
#p1=scatter3D([],0)
fig = plt.figure()
ax = fig.add_subplot(projection="3d")
x = zeros(0)
y = zeros(0)
z = zeros(0)
h = zeros(0)
for i in path
    append!( x, i.state.x )
    append!( y, i.state.y )
    append!( z, 7 )
    append!( h, dirAngle(i.state.heading) )
end
print("x:",x,"\ny:",y,"\nz:",z)
for i in 1:3
    tx = zeros(0)
    ty = zeros(0)
    tz = zeros(0)
    for t in target_trajectories[:,i]
        append!( tx, t.x )
        append!( ty, t.y )
        append!( tz, 0 )
    end
    print("\ntx:",tx,"\nty:",ty,"\ntz:",tz)
    ax.plot(tx,ty,tz)
end

ax.plot(x,y,z)

x:

[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]
y:[1.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0]
z:[7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0]
tx:[7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0]
ty:[7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0]
tz:[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]

LoadError: BoundsError: attempt to access 20×1 Matrix{Target} at index [1:20, 2]

In [ ]:
fov = [2*atan(sensor.resolution[1], 2*focal_length[1]); 2*atan(sensor.resolution[2], 2*focal_length[2])]
print(rad2deg(fov[1]), " ", rad2deg(fov[2]))
print(h)


for i in 1:20
    fig = plt.figure()
    ax = fig.add_subplot(projection="3d")
    location = [path[i].state.x; path[i].state.y; 7]
    camera_heading = [cos(dirAngle(path[i].state.heading)); sin(dirAngle(path[i].state.heading)); -sin(pitch)]
    print(path[i].state.heading, " ", location, " ", location+camera_heading, "\n")
    ax.scatter(path[i].state.x, path[i].state.y, 7, color="red")
    ax.scatter(path[i].state.x+camera_heading[1], path[i].state.y+camera_heading[2], 7+camera_heading[3], color="blue")
    ax.scatter(target_trajectories[i,1].x, target_trajectories[i,1].y, 0)
    ax.scatter(target_trajectories[i,2].x, target_trajectories[i,2].y, 0)
    ax.scatter(target_trajectories[i,3].x, target_trajectories[i,3].y, 0)
end

In [39]:
fov = [2*atan(sensor.resolution[1], 2*focal_length[1]); 2*atan(sensor.resolution[2], 2*focal_length[2])]
print(rad2deg(fov[1]), " ", rad2deg(fov[2]), "\n")
for w in [0, sensor.resolution[1]]
    for h in [0, sensor.resolution[2]]
        point = inv(sensor.extrinsics)*inv(sensor.intrinsics)*[w;h;1]
        print(w, " ", h, " ", point, "\n")
    end
end

82.4002527426068 61.9527999101565
0.0 0.0 [1.0, 0.8754377188594297, 0.6003001500750376]
0.0 2160.0 [1.0, 0.8754377188594297, -0.6003001500750376]
3840.0 0.0 [1.0, -0.8754377188594297, 0.6003001500750376]
3840.0 2160.0 [1.0, -0.8754377188594297, -0.6003001500750376]


In [14]:
fov = [2*atan(sensor.resolution[1], 2*focal_length[1]); 2*atan(sensor.resolution[2], 2*focal_length[2])]
print(rad2deg(fov[1]), " ", rad2deg(fov[2]), "\n")
for w in [0, sensor.resolution[1]]
    for h in [0, sensor.resolution[2]]
        point = inv(sensor.extrinsics)*inv(sensor.intrinsics)*[w;h;1]
        print(w, " ", h, " ", point, "\n")
    end
end

82.4002527426068 61.9527999101565
0.0 0.0 [1.1315830880519089, 0.8754377188594297, -0.2826304743211862]
0.0 2160.0 [0.2826304743211864, 0.8754377188594297, -1.1315830880519087]
3840.0 0.0 [1.1315830880519089, -0.8754377188594297, -0.2826304743211862]
3840.0 2160.0 [0.2826304743211864, -0.8754377188594297, -1.1315830880519087]
